In [1]:
import numpy as np

In [ ]:
import numpy as np
import cvxpy as cp
from scipy.optimize import linprog

def test_assumptions(A, b, tol=1e-6):
    """
    Tests the five assumptions for the optimization problem min ||x||^2 s.t. Ax >= b.

    Args:
        A (np.ndarray): The matrix A of shape (m, d).
        b (np.ndarray): The vector b of shape (m,).
        tol (float): Tolerance for floating-point comparisons.
    """
    m, d = A.shape
    print(f"Testing assumptions for a polyhedron in R^{d} with {m} constraints.\n")

    # --- Find x* and KKT multipliers ---
    # We need to find x* = argmin ||x||^2 subject to Ax >= b. This is a QP.
    x = cp.Variable(d)
    objective = cp.Minimize(cp.sum_squares(x))
    constraints = [A @ x >= b]
    problem = cp.Problem(objective, constraints)
    
    # Solve the problem
    try:
        problem.solve()
        if problem.status not in ["optimal", "optimal_inaccurate"]:
            print("❌ Could not find an optimal x*. Cannot proceed.")
            return
        x_star = x.value
        # KKT multipliers are the dual variables of the constraints
        kkt_multipliers = constraints[0].dual_value
    except Exception as e:
        print(f"❌ An error occurred during optimization: {e}")
        return

    print(f"Found optimal x*: {np.round(x_star, 4)}")
    print("---" * 10)

    # --- Assumption (a): A has full column rank ---
    print("## Assumption (a): The matrix A has full column rank")
    rank_A = np.linalg.matrix_rank(A)
    if rank_A == d:
        print(f"✅ PASSED: rank(A) is {rank_A}, which equals the number of columns (d={d}).")
    else:
        print(f"❌ FAILED: rank(A) is {rank_A}, but should be {d}.")
    print("---" * 10)


    # --- Assumption (b): The polyhedron is full-dimensional ---
    print("## Assumption (b): The polyhedron is full-dimensional")
    # To check this, we find a point x in the strict interior, i.e., Ax > b.
    # We can solve a linear program: max s s.t. Ax - s*1 >= b
    # This is equivalent to min -s s.t. s*1 - Ax <= -b
    # Let variables be z = [x_1, ..., x_d, s].
    c = np.zeros(d + 1)
    c[-1] = -1  # Minimize -s
    A_ub = np.hstack((-A, np.ones((m, 1))))
    b_ub = -b
    res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=(None, None), method='highs')

    if res.success and res.fun < -tol: # if min(-s) < 0 -> max(s) > 0
        print(f"✅ PASSED: Found a point in the strict interior (max slack s = {-res.fun:.4f} > 0).")
    else:
        print("❌ FAILED: Could not find a point in the strict interior. The polyhedron may not be full-dimensional.")
    print("---" * 10)

    # --- Assumption (c): The vector x* satisfies ||x*|| > 0 ---
    print("## Assumption (c): The vector x* satisfies ||x*|| > 0")
    norm_x_star = np.linalg.norm(x_star)
    if norm_x_star > tol:
        print(f"✅ PASSED: ||x*|| = {norm_x_star:.4f}, which is greater than 0.")
    else:
        print(f"❌ FAILED: ||x*|| = {norm_x_star:.4f}. x* is the zero vector.")
    print("---" * 10)


    # --- Assumption (d): Structure of (x*, a_1, ..., a_k) ---
    print("## Assumption (d): The matrix (x*, a_1, ..., a_k) is upper triangular")
    positive_indices = np.where(kkt_multipliers > tol)[0]
    k = len(positive_indices)
    print(f"Found k={k} positive KKT multipliers at indices: {positive_indices}")
    
    if k == 0:
        print("No positive KKT multipliers, so this assumption is vacuously true or not applicable.")
        # If x* is in the interior, k=0. Then the matrix is just (x*).
        # A single column matrix is trivially upper triangular.
        if np.linalg.norm(x_star) < tol:
             print("✅ PASSED: k=0 because x* is the origin, which is in the polyhedron's interior.")
        else:
             print("⚠️  WARNING: k=0 but x* is not the origin. This implies x* is in the interior but not at the origin, which shouldn't happen for this problem.")
    else:
        active_A_rows = A[positive_indices, :]
        # The matrix is formed by x* and the transposes of the rows of A
        # corresponding to positive multipliers, treated as column vectors.
        matrix_d = np.column_stack([x_star] + [row.T for row in active_A_rows])
        
        # Check if it's upper triangular
        is_upper_triangular = np.allclose(matrix_d, np.triu(matrix_d), atol=tol)
        
        if is_upper_triangular:
            print(f"✅ PASSED: The matrix formed by x* and the {k} active constraint vectors is upper triangular.")
        else:
            print(f"❌ FAILED: The matrix is not upper triangular.")
            print("Matrix:\n", np.round(matrix_d, 4))
    print("---" * 10)


    # --- Assumption (e): Condition on inactive/independent vectors ---
    print("## Assumption (e): Existence of u >= k")
    # For each constraint j, check if (a_j^T x* > b_j) OR (a_j is LI from a_1,...,a_k)
    active_A_base = A[positive_indices, :]
    rank_active_A = np.linalg.matrix_rank(active_A_base)
    
    property_P = np.zeros(m, dtype=bool)
    for j in range(m):
        is_strictly_inactive = (A[j, :] @ x_star - b[j]) > tol
        
        # Check if a_j is linearly independent of the active set
        combined_matrix = np.vstack([active_A_base, A[j, :]])
        is_independent = np.linalg.matrix_rank(combined_matrix) > rank_active_A
        
        property_P[j] = is_strictly_inactive or is_independent

    # The assumption requires the boolean array `property_P` to be sorted,
    # i.e., of the form [False, False, ..., True, True].
    is_sorted = all(property_P[i] <= property_P[i+1] for i in range(m - 1))
    
    if not is_sorted:
        print("❌ FAILED: The property P(j) is not sorted over j. No such u exists.")
        print(f"Property P array: {property_P}")
    else:
        # Find the potential value of u
        first_true_idx = np.argmax(property_P)
        if not property_P[first_true_idx]: # This means all are False
            u = m - 1
        else:
            u = first_true_idx - 1
        
        # Check the condition u >= k
        # Note: The assumption probably means u >= k if constraints are 1-indexed.
        # For 0-indexed arrays, this is u+1 >= k.
        if (u + 1) >= k:
            print(f"✅ PASSED: Found u = {u} which satisfies u+1={u+1} >= k={k}.")
            print(f"Property P is False for j <= {u} and True for j > {u}.")
        else:
            print(f"❌ FAILED: Found a potential u = {u}, but it does not satisfy u+1={u+1} >= k={k}.")


In [ ]:
def build_constraints_no_early_exercise_put(S0, K_strike, r, q, sigma, T, b_grid):
    K_grid = len(b_grid)
    # Y = c + H z, with lower-triangular H
    dt   = T / K_grid
    step = sigma * np.sqrt(dt)
    H    = np.tril(np.ones((K_grid, K_grid))) * step
    k    = np.arange(1, K_grid + 1)
    c    = np.log(S0) + (r - q - 0.5 * sigma * sigma) * k * dt
    
    # Use only k = 1..K-1
    G = H[:-1, :-1]
    h = b_grid[:-1] - c[:-1]

   
    return G, h

In [99]:
def make_put_boundary_grid(K, T, K_grid,sigma):

    dt   = T / K_grid
    taus = T - np.arange(1, K_grid + 1) * dt
    s    = K - sigma*np.sqrt(taus)               # there should be a beta before the sigma but I keep it simple
    return np.log(s)
def make_call_boundary_grid(K, T, K_grid,sigma):

    dt   = T / K_grid
    taus = T - np.arange(1, K_grid + 1) * dt
    s    =sigma*np.sqrt(taus) - K          # there should be a beta before the sigma but I keep it simple
    return np.log(s)

In [463]:
S0, K_strike = 154.51, 160.0
r, q, sigma, T = 0.03, 0.0, 1, 1.0
K_grid = 252    # use 252 for daily
delta  = 0.02
alpha  = 0.5
N_tot  = int(1e5)
rng    = np.random.default_rng(12345)

b_grid_put = make_put_boundary_grid(K_strike, T, K_grid, sigma)
A, b = build_constraints_no_early_exercise_put(S0, K_strike, r, q, sigma, T, b_grid_put)
#test_assumptions(A, b)  


In [464]:
print(np.exp(b_grid_put))


[159.0019861  159.00397616 159.0059702  159.00796825 159.00997034
 159.01197648 159.0139867  159.01600103 159.01801949 159.02004211
 159.02206892 159.02409993 159.02613517 159.02817468 159.03021848
 159.0322666  159.03431906 159.03637589 159.03843712 159.04050278
 159.04257289 159.04464749 159.04672661 159.04881027 159.0508985
 159.05299134 159.05508882 159.05719096 159.0592978  159.06140936
 159.06352569 159.06564682 159.06777276 159.06990357 159.07203927
 159.0741799  159.07632549 159.07847607 159.08063169 159.08279237
 159.08495815 159.08712907 159.08930517 159.09148647 159.09367303
 159.09586488 159.09806205 159.10026459 159.10247253 159.10468592
 159.10690479 159.10912919 159.11135916 159.11359474 159.11583597
 159.1180829  159.12033556 159.12259401 159.12485829 159.12712844
 159.12940451 159.13168655 159.1339746  159.13626871 159.13856893
 159.14087531 159.14318789 159.14550674 159.1478319  159.15016341
 159.15250135 159.15484575 159.15719667 159.15955416 159.16191829
 159.164289

In [465]:
### Assumption a ###
d = A.shape[1]
rank_A = np.linalg.matrix_rank(A)
if rank_A == d:
    print(f"✅ PASSED: rank(A) is {rank_A}, which equals the number of columns (d={d}).")
else:
    print(f"❌ FAILED: rank(A) is {rank_A}, but should be {d}.")

✅ PASSED: rank(A) is 251, which equals the number of columns (d=251).


In [466]:
### Assumption b ###
import cvxpy as cp
n,n = A.shape
t = cp.Variable()
x = cp.Variable(d)

constraints = [ 
    A@x >= b + t* np.ones(n)
]

prob = cp.Problem(cp.Maximize(t), constraints)
prob.solve(solver=cp.ECOS)

print("status:", prob.status)
print("t* =", t.value)

### SATISFIED ###

status: unbounded
t* = None


In [467]:
### Assumption c ###

x = cp.Variable(d)

constraints = [
    A@x >= b 
]

objective = cp.Minimize(cp.sum_squares(x))

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.ECOS)

print("status:", prob.status)
print(objective.value)


### SATISFIED ###

status: optimal
0.4594689852968218


In [468]:
### Assumption d ###

import numpy as np
from numpy.linalg import norm
from scipy.linalg import qr

tol = 1e-8



x_star = x.value
lambdas = np.array(constraints[0].dual_value).reshape(-1)  # shape (m,)



# Given: A (m x n), b (m,), x_star (n,), lambdas (m,)
I_pos = [i for i in range(len(b)) if lambdas[i] > tol]


# Build M = [x*, a_i1, ..., a_ik]
cols = [x_star.reshape(-1,1)] + [A[i].reshape(-1,1) for i in I_pos]
M = np.hstack(cols)                      # shape n x (k+1)
p = M.shape[1]

rank_M = np.linalg.matrix_rank(M)
print(f"rank(M) = {rank_M}, number of columns p = {p}")
if rank_M < p:
    print("Condition (d) fails: columns are dependent → cannot get the required triangular form.")


rank(M) = 42, number of columns p = 42


In [469]:
### Assumption e ###

import numpy as np
from numpy.linalg import norm
from scipy.linalg import qr

eps_act = 1e-8
eps_span = 1e-8

# A (m x n), rows a_j^T; b (m,), x_star (n,), lambdas (m,)
m, n = A.shape
a_rows = A  # each row is a_j^T

I_plus = [j for j in range(m) if lambdas[j] > 1e-8]
k = len(I_plus)

# Build Q basis for span{a_i, i in I_plus}
if k > 0:
    B = a_rows[I_plus, :].T         # columns are a_i
    Q, R = qr(B, mode='economic')   # columns of Q span S
else:
    Q = np.zeros((n,0))             # empty basis

def active(j):
    return abs(a_rows[j] @ x_star - b[j]) <= eps_act

def strict_inactive(j):
    return (a_rows[j] @ x_star - b[j]) >= eps_act

def indep_of_S(j):
    if Q.shape[1] == 0:
        return norm(a_rows[j]) > eps_span
    proj = Q @ (Q.T @ a_rows[j])
    return norm(a_rows[j] - proj) > eps_span

def in_span(j):
    return not indep_of_S(j)

J_dep = [j for j in range(m) if (j not in I_plus) and active(j) and in_span(j)]

u = k + len(J_dep)

# Verify (e)
ok_le_u = all(active(j) and in_span(j) for j in (I_plus + J_dep))
rest = [j for j in range(m) if j not in I_plus and j not in J_dep]
ok_gt_u = all(strict_inactive(j) or indep_of_S(j) for j in rest)

print("u =", u, " | condition (e) holds? ", ok_le_u and ok_gt_u)


u = 41  | condition (e) holds?  False


/var/folders/7j/lh0k6xxj3yn_kdygsttx8q5c0000gn/T/ipykernel_42069/1296530667.py:33: RuntimeWarning: divide by zero encountered in matmul
  proj = Q @ (Q.T @ a_rows[j])
/var/folders/7j/lh0k6xxj3yn_kdygsttx8q5c0000gn/T/ipykernel_42069/1296530667.py:33: RuntimeWarning: overflow encountered in matmul
  proj = Q @ (Q.T @ a_rows[j])
/var/folders/7j/lh0k6xxj3yn_kdygsttx8q5c0000gn/T/ipykernel_42069/1296530667.py:33: RuntimeWarning: invalid value encountered in matmul
  proj = Q @ (Q.T @ a_rows[j])


In [460]:
"""
Algorithm 4 — Preprocessing
Inputs: A (m×d), b (m,), mu (d,), Sigma (d×d, SPD)
Outputs: b_new, A_new, x_star_bar, s   (and some useful extras)

Dependencies:
    pip install numpy scipy cvxpy
"""

from __future__ import annotations
import numpy as np
import scipy.linalg as la
import cvxpy as cp
from typing import Dict, Any, Tuple

def _matrix_rank(M: np.ndarray, tol: float = 1e-10) -> int:
    s = np.linalg.svd(M, compute_uv=False)
    return int((s > tol * max(M.shape) * (s[0] if s.size else 1.0)).sum())

def _nullspace_via_svd(M: np.ndarray, tol: float = 1e-10) -> np.ndarray:
    """
    Return a basis for the nullspace of M as columns (shape: n×q),
    using SVD of M = U S V^T and selecting columns of V for zero singular values.
    """
    U, S, Vt = la.svd(M, full_matrices=True)
    n = Vt.shape[1]
    if S.size == 0:
        return np.eye(n)
    r = (S > tol * max(M.shape) * S[0]).sum()
    return Vt[r:].T  # n×(n-r)  (columns are right-singular vectors for zero singular values)

def algorithm4_preprocess(
    A_in: np.ndarray,
    b_in: np.ndarray,
    mu: np.ndarray,
    Sigma: np.ndarray,
    tol_z: float = 1e-9,
    tol_rank: float = 1e-10,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, int, Dict[str, Any]]:
    """
    Implements Algorithm 4 as in the provided figures.

    Returns
    -------
    b_new : (m,)
        Shifted right-hand side after the affine/whitening transforms.
    A_new : (m,d)
        Preprocessed constraint matrix after re-orderings and the final QR rotation.
    x_star_bar : (d,)
        The transformed minimizer after the final QR rotation (step 14).
    s : int
        Rank s := rank(a_1, ..., a_k) after re-ordering (step 11).
    info : dict
        Extra diagnostics: {'L','Q','x_star','lambda_bar','lambda_adj','z','perm','k','u','n'}
    """
    A = np.asarray(A_in, dtype=float).copy()
    b = np.asarray(b_in, dtype=float).copy()
    mu = np.asarray(mu, dtype=float).copy()
    Sigma = np.asarray(Sigma, dtype=float).copy()

    m, d = A.shape
    assert b.shape == (m,)
    assert mu.shape == (d,)
    assert Sigma.shape == (d, d)

    # ---- 2. Cholesky: LL^T = Sigma
    L = la.cholesky(Sigma, lower=True)

    # ---- 3. Shift & whiten: b <- b - A mu ; A <- A L
    b = b - A @ mu
    A = A @ L

    # ---- 4. QP: minimize 0.5 ||x||^2  s.t.  A x >= b
    # Use 'b - A x <= 0' so CVXPY duals are >= 0 and equal to \bar{\lambda}.
    x = cp.Variable(d)
    cons = [b - A @ x <= 0]
    prob = cp.Problem(cp.Minimize(0.5 * cp.sum_squares(x)), cons)
    try:
        prob.solve(solver=cp.ECOS, verbose=False)
    except Exception:
        prob.solve(solver=cp.SCS, verbose=False)

    x_star = x.value
    if x_star is None:
        raise RuntimeError("QP in step 4 failed to solve.")

    lambda_bar_full = cons[0].dual_value  # >= 0
    z_full = (A @ x_star - b)  # >= 0

    # ---- 5. Determine active set n = {i | z_i = 0}. Reorder so first n are active (z_i≈0)
    active_mask = np.abs(z_full) <= tol_z
    n = int(active_mask.sum())
    idx_active = np.flatnonzero(active_mask)
    idx_inactive = np.flatnonzero(~active_mask)
    perm_step5 = np.concatenate([idx_active, idx_inactive])
    A = A[perm_step5, :]
    b = b[perm_step5]
    z_full = z_full[perm_step5]
    lambda_bar_full = lambda_bar_full[perm_step5]

    # convenience slices
    A_act = A[:n, :]        # rows of active constraints
    lam_bar = lambda_bar_full[:n]  # \bar{\lambda}_i, i=1..n

    # ---- 6. SVD of (a1,...,a_n) as columns: build d×n matrix with columns a_i
    # Each a_i is a row of A_act, so columns matrix is A_act.T (d×n).
    M_cols = A_act.T
    Wtilde = _nullspace_via_svd(M_cols, tol=tol_rank)  # shape n×q (q may be 0)

    # ---- 7–8. LP: min y s.t. lam_bar + Wtilde α + y 1 >= 0,  y >= 0
    q = Wtilde.shape[1]
    if q > 0:
        alpha = cp.Variable(q)
        y = cp.Variable()
        cons_lp = [lam_bar + Wtilde @ alpha + y * np.ones(n) >= 0, y >= 0]
        lp = cp.Problem(cp.Minimize(y), cons_lp)
        try:
            lp.solve(solver=cp.ECOS, verbose=False)
        except Exception:
            lp.solve(solver=cp.SCS, verbose=False)
        if alpha.value is None or y.value is None:
            raise RuntimeError("LP in step 8 failed to solve.")
        lam_adj_first_n = lam_bar + Wtilde @ alpha.value
    else:
        # No nullspace; just lift by the smallest nonneg y (which is max(0, -min(lam_bar)))
        y_needed = max(0.0, float(-np.min(lam_bar)))
        lam_adj_first_n = lam_bar + y_needed

    # Full λ after adjustment: first n possibly changed, others (inactive) are 0.
    lambda_adj = np.zeros_like(lambda_bar_full)
    lambda_adj[:n] = lam_adj_first_n
    # sanity nonneg truncation
    lambda_adj = np.maximum(lambda_adj, 0.0)

    # ---- 9. k = number of strictly positive λ_i among first n
    k = int((lam_adj_first_n > 0).sum())

    # ---- 10. Reorder so the first k active constraints are those with λ_i > 0
    pos_idx_local = np.flatnonzero(lam_adj_first_n > 0)
    zero_idx_local = np.flatnonzero(lam_adj_first_n <= 0)
    perm_active10 = np.concatenate([pos_idx_local, zero_idx_local])  # indices in [0..n-1]
    # Apply this reordering within the active block
    A[:n, :] = A[:n, :][perm_active10, :]
    b[:n] = b[:n][perm_active10]
    lambda_adj[:n] = lambda_adj[:n][perm_active10]
    z_full[:n] = z_full[:n][perm_active10]

    # ---- 11. s = rank(a1,...,a_k) (as columns); u = # {j ≤ n | rank(a1,...,a_k, a_j) = s}
    # Build columns matrix for a1..a_k
    if k > 0:
        Ak_cols = A[:k, :].T  # d×k
        s_rank = _matrix_rank(Ak_cols, tol=tol_rank)
    else:
        Ak_cols = np.empty((A.shape[1], 0))
        s_rank = 0

    def in_span_of_Ak(row_vec: np.ndarray) -> bool:
        if k == 0:
            return False
        M = np.column_stack([Ak_cols, row_vec.reshape(-1, 1)])
        return _matrix_rank(M, tol=tol_rank) == s_rank

    dep_mask = []
    for j in range(k, n):
        dep_mask.append(in_span_of_Ak(A[j, :]))
    dep_mask = np.array(dep_mask, dtype=bool)
    u = k + int(dep_mask.sum())

    # ---- 12. Reorder remaining active constraints so that a1..a_u satisfy Assumption 1(e):
    # first k (λ>0), then those among k+1..n that are dependent (span), then independent.
    dep_idx_local = (k + np.flatnonzero(dep_mask))
    indep_idx_local = (k + np.flatnonzero(~dep_mask))
    perm_active12 = np.concatenate([np.arange(k), dep_idx_local, indep_idx_local])
    A[:n, :] = A[:n, :][perm_active12, :]
    b[:n] = b[:n][perm_active12]
    lambda_adj[:n] = lambda_adj[:n][perm_active12]
    z_full[:n] = z_full[:n][perm_active12]

    # ---- 13. QR of (x*, a1, ..., a_k) as columns
    # Build M = [x*, a1, ..., a_k] with each a_i a column
    if k > 0:
        M_for_qr = np.column_stack([x_star, A[:k, :].T])  # d×(k+1)
    else:
        M_for_qr = x_star.reshape(-1, 1)  # d×1

    Q, R = la.qr(M_for_qr, mode="full")  # Q is d×d orthonormal

    # ---- 14. Set x* <- Q^T x*, A <- A Q
    x_star_bar = Q.T @ x_star
    A_new = A @ Q
    b_new = b  # already shifted in step 3

    info = dict(
        L=L, Q=Q, R=R, x_star=x_star, lambda_bar=lambda_bar_full,
        lambda_adj=lambda_adj, z=z_full, k=k, u=u, n=n,
        perm_step5=perm_step5, tol_z=tol_z, tol_rank=tol_rank, s=s_rank,
    )

    return b_new, A_new, x_star_bar, s_rank, info




In [451]:
S0, K_strike = 154.51, 155.0
r, q, sigma, T = 0.03, 0.0, 1, 1.0
K_grid = 252    # use 252 for daily
delta  = 0.02
alpha  = 0.5
N_tot  = int(1e5)
rng    = np.random.default_rng(12345)
mu= np.zeros(K_grid-1) 
Sigma = np.eye(K_grid-1) 
b_grid_put = make_put_boundary_grid(K_strike, T, K_grid, sigma)
A, b = build_constraints_no_early_exercise_put(S0, K_strike, r, q, sigma, T, b_grid_put)
b_new, A_new, x_bar, s, info = algorithm4_preprocess(A, b, mu, Sigma)

/var/folders/7j/lh0k6xxj3yn_kdygsttx8q5c0000gn/T/ipykernel_42069/246570891.py:70: RuntimeWarning: divide by zero encountered in matmul
  b = b - A @ mu
/var/folders/7j/lh0k6xxj3yn_kdygsttx8q5c0000gn/T/ipykernel_42069/246570891.py:70: RuntimeWarning: overflow encountered in matmul
  b = b - A @ mu
/var/folders/7j/lh0k6xxj3yn_kdygsttx8q5c0000gn/T/ipykernel_42069/246570891.py:70: RuntimeWarning: invalid value encountered in matmul
  b = b - A @ mu
/var/folders/7j/lh0k6xxj3yn_kdygsttx8q5c0000gn/T/ipykernel_42069/246570891.py:71: RuntimeWarning: divide by zero encountered in matmul
  A = A @ L
/var/folders/7j/lh0k6xxj3yn_kdygsttx8q5c0000gn/T/ipykernel_42069/246570891.py:71: RuntimeWarning: overflow encountered in matmul
  A = A @ L
/var/folders/7j/lh0k6xxj3yn_kdygsttx8q5c0000gn/T/ipykernel_42069/246570891.py:71: RuntimeWarning: invalid value encountered in matmul
  A = A @ L
/var/folders/7j/lh0k6xxj3yn_kdygsttx8q5c0000gn/T/ipykernel_42069/246570891.py:88: RuntimeWarning: divide by zero enco

In [452]:
test_assumptions(A_new, b_new)

Testing assumptions for a polyhedron in R^251 with 251 constraints.

Found optimal x*: [-0.4718  0.     -0.     -0.     -0.    ]...
------------------------------
## Assumption (a): The matrix A has full column rank
✅ PASSED: rank(A) is 251, which equals the number of columns (d=251).
------------------------------
## Assumption (b): The polyhedron is full-dimensional
❌ FAILED: Could not find a point in the strict interior. The polyhedron may not be full-dimensional.
------------------------------
## Assumption (c): The vector x* satisfies ||x*|| > 0
✅ PASSED: ||x*|| = 0.4718, which is greater than 0.
------------------------------
## Assumption (d): The matrix (x*, a_1, ..., a_k) is upper triangular
Found k=1 positive KKT multipliers at indices: [0]
❌ FAILED: The matrix is not upper triangular.
Matrix sample (first 5x_):
 [[-0.4718 -0.998 ]
 [ 0.     -0.    ]
 [-0.     -0.    ]
 [-0.     -0.    ]
 [-0.     -0.    ]]
------------------------------


/opt/miniconda3/envs/HK_env/lib/python3.13/site-packages/scs/__init__.py:83: UserWarning: Converting A to a CSC (compressed sparse column) matrix; may take a while.
  warn(
/opt/miniconda3/envs/HK_env/lib/python3.13/site-packages/scs/__init__.py:113: UserWarning: Converting P to a CSC (compressed sparse column) matrix; may take a while.
  warn(
